This notebook aims to discuss the upper tropospheric lihgtning NO2 lifetime in the Arctic.

We can estimate the lightning NO2 lifetime by successive orbits with the dissipated convection (no lightning):

$LNO_{2_{T1}}$ * $A_{T1}$ = $LNO_{2_{T0}}$ * $A_{T0}$ * $e^\frac{{-(T_1-T_0)}}{\tau}$

where $T_{<num>}$ is the TROPOMI overpass time,
$LNO_{2_{T<num>}}$ is the $LNO_2$ above cloud in the mask,
$A_{T<num>}$ is the area (m2) of $LNO_{2_{T<num>}}$ pixel,
and $\tau$ is the $NO_2$ lifetime.

Note that the time difference between consecutive swaths is usually 100 min, so the equation can be written as:

$\tau$ = -(100/60) / ln[($LNO_{2_{T1}}$* $A_{T1}$) / ($LNO_{2_{T0}}$ * $A_{T0}$)]

In [1]:
import pandas as pd
import proplot as pplt
import xarray as xr
import numpy as np

## Read picked cases for calculating lifetime

In [2]:
df = pd.read_csv('../data/S5P_LNO2_lifetime.csv') # generated by s5p_lnox_pe_lifetime.py

df['area_diff'] = df.groupby('case')['area'].diff()
df['lno2geo_ratio'] = df['lno2geo'].div(df.groupby('case')['lno2geo'].shift(1))
df['lno2vis_ratio'] = df['lno2vis'].div(df.groupby('case')['lno2vis'].shift(1))
df['lno2_ratio'] = df['lno2'].div(df.groupby('case')['lno2'].shift(1))

df['lno2geo_lifetime'] = -(100/60)/np.log(df['lno2geo_ratio'])
df['lno2vis_lifetime'] = -(100/60)/np.log(df['lno2vis_ratio'])
df['lno2_lifetime'] = -(100/60)/np.log(df['lno2_ratio'])
df['area_percdiff'] = df['area_diff'] / df['area'].shift() * 100

In [3]:
# df_cases = df[(df['nlightning']<=10)&(df['lno2_lifetime']>0)&(df['lno2_lifetime']<24)][['case','time', 'lno2geo', 'lno2vis', 'lno2', 'lno2geo_lifetime', 'lno2vis_lifetime', 'lno2_lifetime', 'area_percdiff']]
df_cases = df[(df['nlightning']<=10)&(df['lno2_lifetime']>0)][['time', 'swath', 'longitude', 'latitude', 'area_percdiff', 'lno2_lifetime']]

In [4]:
df_cases.round(decimals=1)#.to_latex()

,time,swath,longitude,latitude,area_percdiff,lno2_lifetime
2,2019-08-14T22:58,Swath09513,126.0,70.4,2.0,1.7
4,2021-07-11T11:40,Swath19395,-84.5,70.9,28.0,5.2
6,2020-06-09T19:11,Swath13767,162.8,73.0,-43.8,3.6
8,2020-06-15T22:18,Swath13854,155.6,78.7,-4.4,3.4
10,2020-07-15T21:19,Swath14279,109.3,75.4,4.5,2.7


In [5]:
print('Mean lifetime: ', df_cases['lno2_lifetime'].mean())
print('Mean - STD: ', df_cases['lno2_lifetime'].mean() - df_cases['lno2_lifetime'].std())
print('Mean + STD: ', df_cases['lno2_lifetime'].mean() + df_cases['lno2_lifetime'].std())

Mean lifetime:  3.2974081522663012
Mean - STD:  2.009800779830173
Mean + STD:  4.585015524702429
